In [5]:
!pip install pandas
!pip install numpy

  Using cached pandas-2.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 699.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 408.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 816.8 kB/s eta 0:00:00a 0:00:01


In [6]:
import pandas as pd

In [7]:
movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
credits = pd.read_csv('dataset/tmdb_5000_credits.csv')

In [12]:
movies = movies.merge(credits, on='title')

# Data Cleaning
removing irrelevant columns and keeping needful ones

In [18]:
#title genres id keywords overview cast crew
movies = movies[['id','title', 'genres', 'keywords','overview','cast','crew']]

In [33]:
movies.isnull().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [34]:
movies.dropna(inplace=True)

/tmp/ipykernel_8371/3786870272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True)


In [35]:
movies.duplicated().sum()

0

In [23]:
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [47]:
import ast

In [48]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'].lower())
    return l

In [42]:
movies['genres'] = movies['genres'].apply(convert)

ValueError: malformed node or string: ['action', 'adventure', 'fantasy', 'science fiction']

In [44]:
movies['keywords'] = movies['keywords'].apply(convert)

In [49]:
def convert2(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'].lower())
            counter+=1
        else:
            break
    return l


In [51]:
movies['cast'] = movies['cast'].apply(convert2)

In [52]:
movies['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [55]:
def convert3(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l 

In [57]:
movies['crew'] = movies['crew'].apply(convert3)

In [62]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [65]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['overview'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])

In [67]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['overview'] + movies['cast'] + movies['crew']

In [70]:
new_df = movies[['id','title','tags']]

In [71]:
new_df

,id,title,tags
0,19995,Avatar,"[action, adventure, fantasy, sciencefiction, a..."
1,285,Pirates of the Caribbean: At World's End,"[adventure, fantasy, action, adventure, fantas..."
2,206647,Spectre,"[action, adventure, crime, action, adventure, ..."
3,49026,The Dark Knight Rises,"[action, crime, drama, thriller, action, crime..."
4,49529,John Carter,"[action, adventure, sciencefiction, action, ad..."
...,...,...,...
4804,9367,El Mariachi,"[action, crime, thriller, action, crime, thril..."
4805,72766,Newlyweds,"[comedy, romance, comedy, romance, comedy, rom..."
4806,231617,"Signed, Sealed, Delivered","[comedy, drama, romance, tvmovie, comedy, dram..."
4807,126186,Shanghai Calling,[]
